In [ ]:
# | default_exp pointNet

# Imports

In [ ]:
# | export
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# | export
class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)

        self.bn1 = nn.InstanceNorm1d(64, affine=True)
        self.bn2 = nn.InstanceNorm1d(128, affine=True)
        self.bn3 = nn.InstanceNorm1d(1024, affine=True)
        self.bn4 = nn.Identity()
        self.bn5 = nn.Identity()
        # self.bn4 = nn.InstanceNorm1d(512, affine=True)
        # self.bn5 = nn.InstanceNorm1d(256, affine=True)

    def forward(self, x):
        B, K, N = x.size()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2)[0]
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = torch.eye(self.k).flatten().to(x.device)
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

In [ ]:
# | export
class PointNetSeg(nn.Module):
    def __init__(self, num_classes):
        super(PointNetSeg, self).__init__()
        self.input_transform = TNet(k=3)

        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)

        self.bn1 = nn.InstanceNorm1d(64, affine=True)
        self.bn2 = nn.InstanceNorm1d(128, affine=True)
        self.bn3 = nn.InstanceNorm1d(1024, affine=True)

        self.fc1 = nn.Conv1d(1088, 512, 1)
        self.fc2 = nn.Conv1d(512, 256, 1)
        self.fc3 = nn.Conv1d(256, num_classes, 1)

        self.bn4 = nn.InstanceNorm1d(512)
        self.bn5 = nn.InstanceNorm1d(256)

        # self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        B, N, _ = x.size()
        x = x.transpose(2, 1)  # B x 3 x N

        trans = self.input_transform(x)
        x = torch.bmm(trans, x)

        x = F.relu(self.bn1(self.conv1(x)))  # B x 64 x N
        x1 = x
        x = F.relu(self.bn2(self.conv2(x)))  # B x 128 x N
        x = F.relu(self.bn3(self.conv3(x)))  # B x 1024 x N

        x = torch.max(x, 2, keepdim=True)[0]  # B x 1024 x 1
        x = x.repeat(1, 1, N)  # B x 1024 x N

        x = torch.cat([x1, x], 1)  # B x 1088 x N

        x = F.relu(self.bn4(self.fc1(x)))  # B x 512 x N
        # x = F.relu(self.bn5(self.dropout(self.fc2(x))))  # B x 256 x N
        x = F.relu(self.bn5(self.fc2(x)))  # B x 256 x N
        x = self.fc3(x)  # B x num_classes x N

        x = x.transpose(2, 1).contiguous()  # B x N x num_classes
        return x